In [22]:
import tensorflow as tf
import numpy as np
from trade_sim import Market
import tensorflow.contrib as contrib
import os
from multiprocessing import Process, SimpleQueue
import random

In [30]:
class Agent:
    def __init__(self, numberOfCurrencies, timeFrame, sess, initialPortfolio=10000.0):
        self._s = sess
        self.inputT = tf.placeholder(shape=[None, numberOfCurrencies, timeFrame, 3], dtype=tf.float32)
        self.conv1 = tf.layers.conv2d(inputs=self.inputT, filters=20, kernel_size=[1,3])
      #  self.conv1 = tf.nn.depthwise_conv2d(self.inputT, [1,3,3,4], [1,1,1,1], 'SAME')
        self.conv2 = tf.layers.conv2d(inputs=self.conv1, filters=50, kernel_size=[1,8])
        self.conv3 = tf.layers.conv2d(inputs = self.conv2, filters=100, kernel_size=[1,41])
        self.conv4 = tf.layers.conv2d(inputs=self.conv3, filters=150, kernel_size=[1,1])
        self.final = tf.layers.dense(self.conv4, 20000)
        self.final2 = tf.layers.dense(self.final, 1)
        self._allocate = tf.nn.softmax(self.final2, axis=1)
        
        self.priceChanges = tf.placeholder(shape=[None, numberOfCurrencies, 1], dtype=tf.float32)
        
        self.loss = -tf.matmul(tf.matrix_transpose(tf.nn.leaky_relu(tf.log(self.priceChanges), alpha=10)),tf.reshape(self._allocate, [-1, numberOfCurrencies, 1]))
        self.averageLoss = tf.matmul(tf.matrix_transpose(self.priceChanges), 
                                             tf.scalar_mul(tf.constant(initialPortfolio), 
                                               tf.reshape(self._allocate, [-1, numberOfCurrencies, 1])))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        self._train = self.optimizer.minimize(self.loss)
        
    def act(self, observation):
        return self._s.run(self._allocate, feed_dict={self.inputT: observation})
    
    def train_step(self, obs, prices):
        batch_feed = {self.inputT : obs,
                     self.priceChanges: prices
                     }
        _, lossValue = self._s.run([self._train, self.averageLoss], feed_dict=batch_feed)
        return lossValue

In [31]:
def importData(simulator):
    testSim = simulator
    q = SimpleQueue()
    jobs = []
    PERIOD_SIZE = 50
    BATCH_SIZE = 1
    BATCH_COUNT = 1
    BATCH_OFFSET = 101
    dates = testSim.getAllDates()
    index = list(range(BATCH_COUNT))
    feed = []
    threads = 16

    running = False
    count = 0
    while 1:
        if count < threads:
            for i in random.sample(index, threads-count if len(index) >= threads-count else len(index)):
                p = Process(target=testSim.processTimePeriod, args=(q, PERIOD_SIZE, dates, BATCH_SIZE * (i + BATCH_OFFSET) + PERIOD_SIZE, BATCH_SIZE))
                jobs.append(p)
                p.start() 
                index.remove(i)
        count = 0
        for p in jobs:
            if not p.is_alive():
                print('Terminating')
                p.terminate()
                jobs.remove(p)
            else:
                count += 1
        while not q.empty():
            print('Getting')
            feed.append(q.get())
        if count == 0 and len(index) == 0: 
            break
    return feed

In [32]:
def main():
    testSim = Market(['EUR','USD'], os.path.abspath('../Data_Processing/ProcessedData'))
    seeds = [3, 5, 7]
    with tf.Session() as sess:
        tf.set_random_seed(seeds[1])
        test1 = Agent(len(testSim.currencies), 50, sess)
        
        feed = importData(testSim)
        print(feed)
        sess.run(tf.global_variables_initializer())
        prices = []
        batches = []
        
        
            
        for episode in range(10000):
            print("Episode: {}".format(episode))
            index = list(range(len(feed)))
            loss = 0
            count = len(feed)
            while len(index) != 0:
                for i in random.sample(index, 1):
                    with tf.device('/gpu:0'):
                        loss += test1.train_step(feed[i][0], feed[i][1])
                    index.remove(i)
            print(loss/count)
            #print("Loss: {}", loss/count)
 
#         for index in range(len(allDates)):
#             obs.append(testSim.processTimePeriod(50, allDates[index]))
#             prices.append(testSim.processTimePeriod(50, allDates[index]))
            
#             i = index
#             while obsSingle is None and pricesSingle is None:
#                 obsSingle = testSim.processTimePeriod(50, allDates[i])
#                 pricesSingle = testSim.processTimePeriod(50, allDates[i])
#                 i += 1
#         for _ in range(20000):
#             test1.train_step(obs, prices)
#         print(test1.act(obs))
#         print(prices)
#         print(test1._s.run(test1.loss, feed_dict={test1.inputT: obs, test1.priceChanges:prices}))
        

In [33]:
if __name__ == "__main__":
    main()

[[1.        ]
 [1.00003816]]
Done
Getting
Terminating
[([array([[[1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.

[[[10000.382]]]
Episode: 128
[[[10000.382]]]
Episode: 129
[[[10000.382]]]
Episode: 130
[[[10000.382]]]
Episode: 131
[[[10000.382]]]
Episode: 132
[[[10000.382]]]
Episode: 133
[[[10000.382]]]
Episode: 134
[[[10000.382]]]
Episode: 135
[[[10000.382]]]
Episode: 136
[[[10000.382]]]
Episode: 137
[[[10000.382]]]
Episode: 138
[[[10000.382]]]
Episode: 139
[[[10000.382]]]
Episode: 140
[[[10000.382]]]
Episode: 141
[[[10000.382]]]
Episode: 142
[[[10000.382]]]
Episode: 143
[[[10000.382]]]
Episode: 144
[[[10000.382]]]
Episode: 145
[[[10000.382]]]
Episode: 146
[[[10000.382]]]
Episode: 147
[[[10000.382]]]
Episode: 148
[[[10000.382]]]
Episode: 149
[[[10000.382]]]
Episode: 150
[[[10000.382]]]
Episode: 151
[[[10000.382]]]
Episode: 152
[[[10000.382]]]
Episode: 153
[[[10000.382]]]
Episode: 154
[[[10000.382]]]
Episode: 155
[[[10000.382]]]
Episode: 156
[[[10000.382]]]
Episode: 157
[[[10000.382]]]
Episode: 158
[[[10000.382]]]
Episode: 159
[[[10000.382]]]
Episode: 160
[[[10000.382]]]
Episode: 161
[[[10000.382]]

[[[10000.382]]]
Episode: 422
[[[10000.382]]]
Episode: 423
[[[10000.382]]]
Episode: 424
[[[10000.382]]]
Episode: 425
[[[10000.382]]]
Episode: 426
[[[10000.382]]]
Episode: 427
[[[10000.382]]]
Episode: 428
[[[10000.382]]]
Episode: 429
[[[10000.382]]]
Episode: 430
[[[10000.382]]]
Episode: 431
[[[10000.382]]]
Episode: 432
[[[10000.382]]]
Episode: 433
[[[10000.382]]]
Episode: 434
[[[10000.382]]]
Episode: 435
[[[10000.382]]]
Episode: 436
[[[10000.382]]]
Episode: 437
[[[10000.382]]]
Episode: 438
[[[10000.382]]]
Episode: 439
[[[10000.382]]]
Episode: 440
[[[10000.382]]]
Episode: 441
[[[10000.382]]]
Episode: 442
[[[10000.382]]]
Episode: 443
[[[10000.382]]]
Episode: 444
[[[10000.382]]]
Episode: 445
[[[10000.382]]]
Episode: 446
[[[10000.382]]]
Episode: 447
[[[10000.382]]]
Episode: 448
[[[10000.382]]]
Episode: 449
[[[10000.382]]]
Episode: 450
[[[10000.382]]]
Episode: 451
[[[10000.382]]]
Episode: 452
[[[10000.382]]]
Episode: 453
[[[10000.382]]]
Episode: 454
[[[10000.382]]]
Episode: 455
[[[10000.382]]

[[[10000.382]]]
Episode: 716
[[[10000.382]]]
Episode: 717
[[[10000.382]]]
Episode: 718
[[[10000.382]]]
Episode: 719
[[[10000.382]]]
Episode: 720
[[[10000.382]]]
Episode: 721
[[[10000.382]]]
Episode: 722
[[[10000.382]]]
Episode: 723
[[[10000.382]]]
Episode: 724
[[[10000.382]]]
Episode: 725
[[[10000.382]]]
Episode: 726
[[[10000.382]]]
Episode: 727
[[[10000.382]]]
Episode: 728
[[[10000.382]]]
Episode: 729
[[[10000.382]]]
Episode: 730
[[[10000.382]]]
Episode: 731
[[[10000.382]]]
Episode: 732
[[[10000.382]]]
Episode: 733
[[[10000.382]]]
Episode: 734
[[[10000.382]]]
Episode: 735
[[[10000.382]]]
Episode: 736
[[[10000.382]]]
Episode: 737
[[[10000.382]]]
Episode: 738
[[[10000.382]]]
Episode: 739
[[[10000.382]]]
Episode: 740
[[[10000.382]]]
Episode: 741
[[[10000.382]]]
Episode: 742
[[[10000.382]]]
Episode: 743
[[[10000.382]]]
Episode: 744
[[[10000.382]]]
Episode: 745
[[[10000.382]]]
Episode: 746
[[[10000.382]]]
Episode: 747
[[[10000.382]]]
Episode: 748
[[[10000.382]]]
Episode: 749
[[[10000.382]]

[[[10000.382]]]
Episode: 1008
[[[10000.382]]]
Episode: 1009
[[[10000.382]]]
Episode: 1010
[[[10000.382]]]
Episode: 1011
[[[10000.382]]]
Episode: 1012
[[[10000.382]]]
Episode: 1013
[[[10000.382]]]
Episode: 1014
[[[10000.382]]]
Episode: 1015
[[[10000.382]]]
Episode: 1016
[[[10000.382]]]
Episode: 1017
[[[10000.382]]]
Episode: 1018
[[[10000.382]]]
Episode: 1019
[[[10000.382]]]
Episode: 1020
[[[10000.382]]]
Episode: 1021
[[[10000.382]]]
Episode: 1022
[[[10000.382]]]
Episode: 1023
[[[10000.382]]]
Episode: 1024
[[[10000.382]]]
Episode: 1025
[[[10000.382]]]
Episode: 1026
[[[10000.382]]]
Episode: 1027
[[[10000.382]]]
Episode: 1028
[[[10000.382]]]
Episode: 1029
[[[10000.382]]]
Episode: 1030
[[[10000.382]]]
Episode: 1031
[[[10000.382]]]
Episode: 1032
[[[10000.382]]]
Episode: 1033
[[[10000.382]]]
Episode: 1034
[[[10000.382]]]
Episode: 1035
[[[10000.382]]]
Episode: 1036
[[[10000.382]]]
Episode: 1037
[[[10000.382]]]
Episode: 1038
[[[10000.382]]]
Episode: 1039
[[[10000.382]]]
Episode: 1040
[[[10000.3

KeyboardInterrupt: 